In [3]:
import pandas as pd
import numpy as np
from rapidfuzz import process, fuzz
import re

In [6]:
mobo_ppp = pd.read_csv('data/raw/ppp/motherboard.csv')
mobo_local = pd.read_csv('data/filtered/prices_motherboard.csv')

In [12]:
mobo_local.head()
mobo_ppp.head()

,name,price,socket,form_factor,max_memory,memory_slots,color
0,Asus PRIME B650-PLUS WIFI,159.99,AM5,ATX,192,4,Black / Silver
1,MSI B650 GAMING PLUS WIFI,169.99,AM5,ATX,192,4,Black
2,MSI MAG B650 TOMAHAWK WIFI,189.42,AM5,ATX,256,4,Black
3,Gigabyte X870E AORUS ELITE WIFI7,325.31,AM5,ATX,256,4,Black
4,Asus PRIME B550M-A WIFI II,99.99,AM4,Micro ATX,128,4,Blue / Silver


filter ppp mobo based on generation

In [21]:
valid_sockets = ['LGA1200', 'LGA1700', 'LGA1851', 'AM4', 'AM5']
filtered_ppp_mobo = mobo_ppp[mobo_ppp['socket'].isin(valid_sockets)]
filtered_ppp_mobo = filtered_ppp_mobo.drop_duplicates(subset='name')

# Save filtered data
filtered_ppp_mobo.to_csv('data/filtered/ppp_filtered_mobo.csv', index=False)



In [19]:
local_parts = mobo_local['item_name'].str.split(', ', expand=True)
local_parts = local_parts.apply(lambda x: x.str.strip())

local_parts.head()

,0,1,2,3,4,5
0,Asus ROG Strix H370-F Gaming,LGA1151,H370,ATX,4*ddr4,None
1,Asus Prime H510M-K R2.0,LGA1200,mATX,2*ddr4,None,None
2,MSI Pro H510M-B,LGA1200,mATX,2*ddr4,None,None
3,Asus Prime H610M-D D4,LGA1700,mATX,2*ddr4,None,None
4,Asus Prime H610M-K D4,LGA1700,mATX,2*ddr4,None,None


In [23]:
mobo_local['model'] = local_parts[0]
mobo_local['socket'] = local_parts[1]
mobo_local['form_factor'] = local_parts[2]
mobo_local['ram_type'] = local_parts[3]

mobo_local.head()

,item_name,category_name,item_price_in_php,stocks,model,socket,form_factor,ram_type
0,"Asus ROG Strix H370-F Gaming, LGA1151, H370, A...",Motherboard,5999.0,34,Asus ROG Strix H370-F Gaming,LGA1151,H370,ATX
1,"Asus Prime H510M-K R2.0, LGA1200, mATX, 2*ddr4",Motherboard,4225.0,11,Asus Prime H510M-K R2.0,LGA1200,mATX,2*ddr4
2,"MSI Pro H510M-B, LGA1200, mATX, 2*ddr4",Motherboard,3450.0,31,MSI Pro H510M-B,LGA1200,mATX,2*ddr4
3,"Asus Prime H610M-D D4, LGA1700, mATX, 2*ddr4",Motherboard,4999.0,8,Asus Prime H610M-D D4,LGA1700,mATX,2*ddr4
4,"Asus Prime H610M-K D4, LGA1700, mATX, 2*ddr4",Motherboard,4880.0,9,Asus Prime H610M-K D4,LGA1700,mATX,2*ddr4


In [33]:
# Normalize the columns
mobo_ppp['name_clean'] = mobo_ppp['name'].str.lower().str.strip()
mobo_local['model_clean'] = mobo_local['model'].str.lower().str.strip()

# Merge on the cleaned keys
merged_mobo = pd.merge(
    mobo_ppp,
    mobo_local[['model_clean', 'item_price_in_php']],
    left_on='name_clean',
    right_on='model_clean',
    how='inner'
)
merged_mobo = merged_mobo.drop(columns=['name_clean', 'model_clean', 'price'])
merged_mobo = merged_mobo.drop_duplicates(subset='name', keep='first')

# Save the merged data
merged_mobo.to_csv('data/processed/mobo_merged.csv')
merged_mobo.head()



,name,socket,form_factor,max_memory,memory_slots,color,item_price_in_php
0,MSI B650 GAMING PLUS WIFI,AM5,ATX,192,4,Black,12350.0
2,MSI MAG B650 TOMAHAWK WIFI,AM5,ATX,256,4,Black,13950.0
4,Gigabyte X870E AORUS ELITE WIFI7,AM5,ATX,256,4,Black,19699.0
6,Asus PRIME B550M-A WIFI II,AM4,Micro ATX,128,4,Blue / Silver,5999.0
8,Gigabyte B650 EAGLE AX,AM5,ATX,192,4,Gray / Black,11199.0


In [36]:
def extract_chipset(name):
    name = name.upper()
    intel_match = re.search(r'(H\d{3}|B\d{3}|Z\d{3})', name)
    amd_match = re.search(r'(A\d{3}|B\d{3}|X\d{3})', name)
    if intel_match:
        return intel_match.group(0)
    elif amd_match:
        return amd_match.group(0)
    return 'Unknown'

merged_mobo['chipset'] = merged_mobo['name'].apply(extract_chipset)

merged_mobo.head()
merged_mobo.to_csv('data/processed/mobo_merged.csv')


In [39]:
def detect_ram_type(row):
    socket = row['socket'].upper()
    name = row['name'].upper()
    
    if socket in ['LGA1200']:
        return 'DDR4'
    elif socket in ['LGA1700']:
        return 'DDR4' if 'DDR4' in name else 'DDR5'
    elif socket in ['LGA1851']:
        return 'DDR5'
    elif socket in ['AM4']:
        return 'DDR4'
    elif socket in ['AM5']:
        return 'DDR5'
    return 'Unknown'

merged_mobo['ram_type'] = merged_mobo.apply(detect_ram_type, axis=1)

merged_mobo.head()
merged_mobo.to_csv('data/processed/mobo_merged.csv')
